# A quick jupyter notebook to experiment with the FastF1 API endpoints and check data format

### Here's a link to the FastF1 github: https://github.com/theOehrly/Fast-F1

In [ ]:
import fastf1
import json

fastf1.Cache.enable_cache("cache")
session = fastf1.get_session(2025, "Suzuka", "Q")
session.load()

In [2]:
suzuka_circuit_info = session.get_circuit_info()
# suzuka_circuit_info.corners

In [3]:
data = {
    "rotation": suzuka_circuit_info.rotation,
    "corners": suzuka_circuit_info.corners.to_numpy().tolist()
}
with open("suzuka_track_info.json", "w") as f:
    json.dump(data, f, indent=2)

In [4]:
best_laps = {}
best_laps["VER"] = session.laps.pick_drivers("VER").pick_fastest()
# best_laps["VER"]

In [5]:
best_laps["PIA"] = session.laps.pick_drivers("PIA").pick_fastest()
# best_laps["PIA"]

In [6]:
tel = best_laps["PIA"].get_telemetry()
print(tel.keys())

Index(['Date', 'SessionTime', 'DriverAhead', 'DistanceToDriverAhead', 'Time',
       'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
       'Distance', 'RelativeDistance', 'Status', 'X', 'Y', 'Z'],
      dtype='object')


# Lets actually draw the lap to see how it looks

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import splprep, splev
import numpy as np

In [11]:
with open("ver_suzuka_2025.json") as f:
    data_ver = json.load(f)
    df_ver = pd.DataFrame(data_ver["location"])

with open("pia_suzuka_2025.json") as f:
    data_pia = json.load(f)
    df_pia = pd.DataFrame(data_pia["location"])

In [12]:
ver_points = np.array(list(zip(df_ver[0].values, df_ver[1].values))).T
ver_tck, ver_u = splprep(ver_points, s=0)

pia_points = np.array(list(zip(df_pia[0].values, df_pia[1].values))).T
pia_tck, pia_u = splprep(pia_points, s=0)


In [ ]:
# Evaluate spline at many points for smoothness
u_fine = np.linspace(0, 1, 5000)
ver_x_smooth, ver_y_smooth = splev(u_fine, ver_tck)
pia_x_smooth, pia_y_smooth = splev(u_fine, pia_tck)

# Plot
plt.figure(figsize=(10,5))
plt.plot(ver_x_smooth, ver_y_smooth, color="blue", label="Verstappen line")
plt.plot(pia_x_smooth, pia_y_smooth, color="orange", label="Piastri line")

plt.xlabel("X")
plt.ylabel("Y")
plt.axis("equal")  # keep aspect ratio correct
plt.title("Driver Lines")
plt.grid(True)
plt.legend()
plt.show()


In [48]:
data = {
    "driver": "VER",
    "lap_time": float(best_laps["VER"]["LapTime"].total_seconds()),
    "distance": tel["Distance"].tolist(),
    "speed": tel["Speed"].tolist(),
    "throttle": tel["Throttle"].tolist(),
    "brake": tel["Brake"].tolist(),
    "location": list(zip(tel["X"].tolist(), tel["Y"].tolist(), tel["Z"].tolist())),
    "n_points": len(tel["Distance"]),
    "smooth_location": list(zip(ver_x_smooth, ver_y_smooth)),
}

with open("ver_suzuka_2025.json", "w") as f:
    json.dump(data, f, indent=2)